In [ ]:
#Importing useful libraries
import numpy as np
import pandas as pd 
import os
import optuna
import pickle
import random

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import  Input, Dense, Dropout, Attention

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, StandardScaler
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, f1_score
from sklearn.utils import class_weight

In [ ]:
#Select which dataset to load
dataset = "rosmap"

#Dataset Loading
if  dataset == "rosmap":
    
    path = "../ROSMAP/"
    #Loading data already early concatenated
    rosmap = pd.read_csv(path + "rosmap_all_omics.csv", index_col = 0)
    r_lab = pd.read_csv(path + "rosmap_labels.csv", index_col = 0)
    X = rosmap.to_numpy()
    y = r_lab
    #Loading specific data split
    omic1_tr = pd.read_csv(path + "1_tr.csv", header = None)
    omic1_te = pd.read_csv(path + "1_te.csv", header = None)
    omic2_tr = pd.read_csv(path + "2_tr.csv", header = None)
    omic2_te = pd.read_csv(path + "2_te.csv", header = None)
    omic3_tr = pd.read_csv(path + "3_tr.csv", header = None)
    omic3_te = pd.read_csv(path + "3_te.csv", header = None)
    lab_tr = pd.read_csv(path + "labels_tr.csv", header = None)
    lab_te = pd.read_csv(path + "labels_te.csv", header = None)
    omic_tr = pd.concat([omic1_tr,omic2_tr], axis = 1)
    omic_tr = pd.concat([omic_tr,omic3_tr], axis = 1)
    omic_te = pd.concat([omic1_te,omic2_te], axis = 1)
    omic_te = pd.concat([omic_te,omic3_te], axis = 1)
    num_classes = 2
        
elif "brca": 
    
    path = "../BRCA/"
    #Loading data already early concatenated
    brca = pd.read_csv(path + "/brca_all_omics.csv", index_col = 0)
    b_lab = pd.read_csv(path + "/brca_labels.csv", index_col = 0)
    #Loading specific data split
    omic1_tr = pd.read_csv(path + "1_tr.csv", header = None)
    omic1_te = pd.read_csv(path + "1_te.csv", header = None)
    omic2_tr = pd.read_csv(path + "2_tr.csv", header = None)
    omic2_te = pd.read_csv(path + "2_te.csv", header = None)
    omic3_tr = pd.read_csv(path + "3_tr.csv", header = None)
    omic3_te = pd.read_csv(path + "3_te.csv", header = None)
    lab_tr = pd.read_csv(path + "labels_tr.csv", header = None)
    lab_te = pd.read_csv(path + "labels_te.csv", header = None)
    omic_tr = pd.concat([omic1_tr,omic2_tr], axis = 1)
    omic_tr = pd.concat([omic_tr,omic3_tr], axis = 1)
    omic_te = pd.concat([omic1_te,omic2_te], axis = 1)
    omic_te = pd.concat([omic_te,omic3_te], axis = 1)
    X = brca.to_numpy()
    y = b_lab
    num_classes = 5

y = y.to_numpy()
y = y.reshape(y.shape[0],)

#If random split == 0, uses the author's split
random_split = 1


#seeds = [30, 300, 3000, 30000, 300000]

In [ ]:
#Create 1000 seeds to perform splitting on data
seeds = []
for i in range(0,1000):
    n = random.randint(1,500000)
    seeds.append(n)

In [ ]:
if dataset == "rosmap":    
    #Binary Classification

    acc = []
    auc = []
    f1 = []

    #Hyperparameters for ROSMAP
    
    drop_rate = 0.3    # 0.5
    drop_rate_out = 0.3
    neuron_dense1 = 2000  #1000
    neuron_dense2 = 40  #160
    neuron_dense3 = 4  #8
    activation_dense = 'tanh'  #relu
    activation_output = 'sigmoid'
    num_epochs = 100
    es_patience = 15
    batch = 64
    val_split = 0.1
    test_split = 0.2
    i = 0

    for seed in seeds:
        
        if i%100 == 0:
            print("___________________________" + str(i) + "_________________")
            
        if random_split:
            
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_split, \
                                                            random_state=seed, stratify = y)
        
        else:
            
            X_train = omic_tr.to_numpy()
            X_test = omic_te.to_numpy()
            y_train = lab_tr.to_numpy().reshape(lab_tr.shape[0],)
            y_test = lab_te.to_numpy().reshape(lab_te.shape[0],)

        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        #Simple Network Architecture
        
        input_layer = Input(shape=(X_train.shape[1],))
        attention = Attention()([input_layer, input_layer])
        x = Dropout(drop_rate)(attention)
        x = Dense(neuron_dense1, activation = activation_dense)(x)
        x = Dropout(drop_rate)(x)
        x = Dense(neuron_dense2, activation = activation_dense)(x)
        x = Dropout(drop_rate_out)(x)
        x = Dense(neuron_dense3, activation = activation_dense)(x)
        output_layer = Dense(1, activation = activation_output)(x)


        model = Model(inputs = input_layer, outputs = output_layer)

        model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
        callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = es_patience)
        history = model.fit(X_train, y_train, epochs = num_epochs, batch_size = batch, \
                  callbacks = [callback], validation_split = val_split, verbose = 0)

        y_pred_ = model.predict(X_test)
        y_pred = np.round(y_pred_)
        y_pred = y_pred.astype(int)

        acc.append(accuracy_score(y_test, y_pred))
        auc.append(roc_auc_score(y_test, y_pred_))
        f1.append(f1_score(y_test, y_pred))
        
        i += 1

    print("ACC:", format(np.mean(acc), ".4f"), format(np.std(acc), ".4f"))
    print("AUC:", format(np.mean(auc), ".4f"), format(np.std(auc), ".4f"))
    print("F1:", format(np.mean(f1), ".4f"), format(np.std(f1), ".4f"))
    
else:
    
    #Multiclass classification
    
    acc = []
    f1m = []
    f1w = []
    
    #Hyperparameters for BRCA
    
    drop_rate = 0.3
    drop_rate_out = 0.3
    neuron_dense1 = 1000
    neuron_dense2 = 160
    neuron_dense3 = 8
    activation_dense = 'relu'
    activation_output = 'softmax'
    num_epochs = 100
    es_patience = 50
    batch = 64
    val_split = 0.1
    test_split = 0.2
        
    for seed in seeds:

        if random_split:
            
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_split, \
                                                            random_state=seed, stratify = y)
        
        else:
            X_train = omic_tr.to_numpy()
            X_test = omic_te.to_numpy()
            y_train = lab_tr.to_numpy().reshape(lab_tr.shape[0],)
            y_test = lab_te.to_numpy().reshape(lab_te.shape[0],)

        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        input_layer = Input(shape=(X_train.shape[1],))
        attention = Attention()([input_layer, input_layer])
        x = Dropout(drop_rate)(attention)
        x = Dense(neuron_dense1, activation = activation_dense)(x)
        x = Dropout(drop_rate)(x)
        x = Dense(neuron_dense2, activation = activation_dense)(x)
        x = Dropout(drop_rate_out)(x)
        x = Dense(neuron_dense3, activation = activation_dense)(x)
        output_layer = Dense(num_classes, activation = activation_output)(x)

        model = Model(inputs = input_layer, outputs = output_layer)

        model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
        callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = es_patience)
        class_weights = class_weight.compute_class_weight(class_weight = 'balanced', \
                                                          classes = np.unique(y_train), y = y_train)
        model.fit(X_train, y_train, epochs = num_epochs,  batch_size = batch, \
                  callbacks=[callback], validation_split = val_split, \
                  class_weight=dict(enumerate(class_weights)),\
                  verbose = 0)

        y_pred = model.predict(X_test)
        y_pred_classes = np.argmax(y_pred, axis=1)

        acc.append(accuracy_score(y_test, y_pred_classes))
        f1w.append(f1_score(y_test, y_pred_classes, average = 'weighted'))
        f1m.append(f1_score(y_test, y_pred_classes, average = 'macro'))

    print("ACC:", format(np.mean(acc), ".4f"), format(np.std(acc), ".4f"))
    print("F1M:", format(np.mean(f1m), ".4f"), format(np.std(f1m), ".4f"))
    print("F1W:", format(np.mean(f1w), ".4f"), format(np.std(f1w), ".4f"))


In [ ]:
#Saving Results
if dataset == "rosmap":
    dic = {"Acc": acc, "Auc": auc, "F1" : f1}

    with open('rosmap_att_early_1000.pickle', 'wb') as handle:
        pickle.dump(dic, handle, protocol=pickle.HIGHEST_PROTOCOL)
elif "brca":
    dic = {"Acc": acc, "F1M": f1m, "F1W" : f1w}

    with open('brca_att_early_1000.pickle', 'wb') as handle:
        pickle.dump(dic, handle, protocol=pickle.HIGHEST_PROTOCOL)